In [2]:
from pyspark import SparkContext,SparkConf
import pyspark
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructField,StringType,DoubleType,IntegerType
from pyspark.sql import functions as f
from pyspark.sql.functions import lit,trim,concat,coalesce,udf,struct
from pyspark.sql import SQLContext


In [3]:
import psutil

p= psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

In [4]:
#sc.stop()
sc = SparkContext('local','similarity')

In [3]:
sc = SparkContext('local','similarity')
sc.setSystemProperty('spark.executor.memory', '4g')
sc.setSystemProperty('spark.executor.cores', '2')
sc.setSystemProperty('spark.driver.maxResultSize','2g')
sc.setSystemProperty('spark.driver.memory','2g')

sc.getConf().getAll() 


[('spark.master', 'local'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.cores', '2'),
 ('spark.app.id', 'local-1556462639267'),
 ('spark.app.name', 'similarity'),
 ('spark.driver.port', '62361'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '2g'),
 ('spark.driver.maxResultSize', '2g'),
 ('spark.driver.host', 'DESKTOP-5MQFV1T'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [5]:
%%time
sqlContext = SQLContext(sc)

Wall time: 248 ms


In [6]:
%%time
#reading the input


input_1 = sc.textFile('./data1.txt')

Wall time: 1.73 s


In [7]:
%%time
input_f=input_1.map(lambda x: x+'~ ') #delimiting every document by ~

Wall time: 0 ns


In [8]:
%%time
input_flat = input_f.flatMap(lambda x: x.split(' '))


input_dist = input_flat.distinct()


input_filter = input_dist.filter(lambda x: x is not None and x!="")


Wall time: 300 ms


In [9]:
%%time
count_non_zero=udf(lambda row:len([x for x in row if x!=None]),IntegerType()) #udf to get count of non_zero columns
sqr=udf(lambda row:(sum([x**2 for x in row if x!=None]))**(1/2),DoubleType()) #udf to get sqrt_of_sum 

Wall time: 0 ns


In [10]:
%%time
#calculating tf
data=input_flat.collect()

dffilt = sqlContext.createDataFrame(input_filter.collect(), StringType())
for i in range(input_1.count()): 
    d1=data.index('~')
    #print(i,d1)
    df = sqlContext.createDataFrame(data[0:d1], StringType())
    data=data[d1+1:]
    df_1=df.withColumn('d'+str(i),f.lit(1)) #adding col for each documnet
    #calculating tf 
    df_f=df_1.groupBy('value').agg(f.round((f.count('d'+str(i))/d1),3).alias('d'+str(i))) #rounding off data to 3 decimal places
    dffilt=dffilt.join(df_f,"value",'left') #adding each col to main dataframe
    


Wall time: 12.2 s


In [11]:
%%time
dffilt.show()

+------------+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+
|       value|   d0|   d1|   d2|   d3|   d4|   d5|   d6|  d7|   d8|   d9|
+------------+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+
|         few| null|0.005| null| null| null| null| null|null| null| null|
|       input| null| null| null| null| null| null| null|null| null|0.007|
| interaction|0.012| null|0.023| null| null| null| null|null| null| null|
|       still| null| null| null| null| null| null| null|null|0.006| null|
|       those|0.012|0.005| null| null| null| null| null|null| null| null|
|     include|0.012| null| null| null| null| null| null|null| null| null|
|  plasticity| null| null|0.008|0.009| null|0.046| null|null|0.006|0.007|
|         map| null|0.005| null| null| null| null| null|null|0.006| null|
|      saline| null| 0.01| null| null|0.005| null| null|null| null| null|
|   extension| null| null| null| null| null|0.011| null|null| null| null|
|      intact| null| null| null| null|

In [12]:
%%time
dffilt=dffilt.withColumn('count',count_non_zero(struct([dffilt[x] for x in dffilt.columns]))-1) #counting non_zero columns and storing in count

Wall time: 230 ms


In [13]:
%%time
dffilt=dffilt.fillna(0) #replace null with 0

Wall time: 61.8 ms


In [14]:
%%time
#calculating IDF
for i in range(input_1.count()): 
    dffilt=dffilt.withColumn('d'+str(i),f.round(f.col('d'+str(i))*f.log(input_1.count()/f.col('count')),3)) #rounding off data to 3 decimal places

Wall time: 6.06 s


In [15]:
%%time
dffilt.show()

+------------+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+
|       value|   d0|   d1|   d2|   d3|   d4|   d5|   d6|  d7|   d8|   d9|count|
+------------+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+
|         few|  0.0|0.012|  0.0|  0.0|  0.0|  0.0|  0.0| 0.0|  0.0|  0.0|    1|
|       input|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0| 0.0|  0.0|0.016|    1|
| interaction|0.019|  0.0|0.037|  0.0|  0.0|  0.0|  0.0| 0.0|  0.0|  0.0|    2|
|       still|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0| 0.0|0.014|  0.0|    1|
|       those|0.019|0.008|  0.0|  0.0|  0.0|  0.0|  0.0| 0.0|  0.0|  0.0|    2|
|     include|0.028|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0| 0.0|  0.0|  0.0|    1|
|  plasticity|  0.0|  0.0|0.006|0.006|  0.0|0.032|  0.0| 0.0|0.004|0.005|    5|
|         map|  0.0|0.008|  0.0|  0.0|  0.0|  0.0|  0.0| 0.0| 0.01|  0.0|    2|
|      saline|  0.0|0.016|  0.0|  0.0|0.008|  0.0|  0.0| 0.0|  0.0|  0.0|    2|
|   extension|  0.0|  0.0|  0.0|  0.0|  

In [16]:
%%time
dffilt=dffilt.where(f.col('value') != '~') #removing ~

Wall time: 62.8 ms


In [17]:
%%time
#Testing data for two words "few" and "input"

dffilt_test=dffilt.where((f.col('value') == 'few') | (f.col('value') == 'input'))

Wall time: 14.4 ms


In [18]:
%%time
dffilt_test.show()

+-----+---+-----+---+---+---+---+---+---+---+-----+-----+
|value| d0|   d1| d2| d3| d4| d5| d6| d7| d8|   d9|count|
+-----+---+-----+---+---+---+---+---+---+---+-----+-----+
|  few|0.0|0.012|0.0|0.0|0.0|0.0|0.0|0.0|0.0|  0.0|    1|
|input|0.0|  0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.016|    1|
+-----+---+-----+---+---+---+---+---+---+---+-----+-----+

Wall time: 1min 51s


In [19]:
%%time
cols=dffilt_test.columns
cols.remove('value')
cols.remove('count')
dffilt_test=dffilt_test.drop('value')
dffilt_test=dffilt_test.drop('count')
#Removed not needed columns
dffilt_test2=dffilt_test.withColumn('sqr',sqr(struct([dffilt_test[x] for x in cols]))) #getting denominator

Wall time: 56.9 ms


In [20]:
%%time
dffilt_test2.show()

+---+-----+---+---+---+---+---+---+---+-----+-----+
| d0|   d1| d2| d3| d4| d5| d6| d7| d8|   d9|  sqr|
+---+-----+---+---+---+---+---+---+---+-----+-----+
|0.0|0.012|0.0|0.0|0.0|0.0|0.0|0.0|0.0|  0.0|0.012|
|0.0|  0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.016|0.016|
+---+-----+---+---+---+---+---+---+---+-----+-----+

Wall time: 3min 17s


In [21]:
%%time
transposed_df = sqlContext.createDataFrame(dffilt_test2.toPandas().transpose().reset_index()) #getting transpose

Py4JJavaError: An error occurred while calling o611.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 42 in stage 218.0 failed 1 times, most recent failure: Lost task 42.0 in stage 218.0 (TID 533, localhost, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:148)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:76)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:86)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:73)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1.apply(EvalPythonExec.scala:131)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1.apply(EvalPythonExec.scala

In [22]:
%%time
transposed_df.show()

NameError: name 'transposed_df' is not defined

In [23]:
%%time
transposed_df=transposed_df.withColumn("mult",f.col('0')*f.col('1')) #getting final values of numerator and denominator(sqr)

NameError: name 'transposed_df' is not defined

In [24]:
%%time
transposed_df.show()

NameError: name 'transposed_df' is not defined

In [25]:
%%time
#dividing numerator and denominator

answer=transposed_df.where(f.col('index') != 'sqr').agg(f.sum('mult')).rdd.collect()[0][0]/transposed_df.select('mult').where(f.col('index') == 'sqr').rdd.collect()[0][0]

NameError: name 'transposed_df' is not defined

In [26]:
answer

NameError: name 'answer' is not defined